<a href="https://colab.research.google.com/github/pauhidalgoo/TDR-MusicNN/blob/main/TDR_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TDR-B**


---



*Segona versió*

Pau Hidalgo Pujol

Aquest és un codi fruit del meu [TDR](https://drive.google.com/file/d/1ZhvwYO2UMx40ZbBRD95WKq1T5hDV4oAG/view?usp=sharing), que se centra en la creació d'una xarxa neuronal per a la composició de música, que és el programa que es troba a continuació. Aquesta és una de les 5 versions creades. Els resultats es poden veure a la següent [carpeta](https://drive.google.com/drive/folders/1ntoY9EntQ5Whj1ASacY-D-gr6cjgxOzm?usp=sharing).

Per executar els  codis, cal clicar el botó que tenen a l'esquerra entre [ ]. 

In [ ]:
#@title Inicialització
#@markdown Aquest codi instal·la la llibreria MIDO. La resta de llibreries que utilitzarem ja venen per defecte en el Colab.

!pip install mido

     |████████████████████████████████| 51 kB 3.3 MB/s 


In [ ]:
#@title Accedeix als arxius de Drive
#@markdown Munta els arxius de Drive de forma que puguin ser accedits pel codi. Et demana un codi d'autorització, que obtens clicant al link i iniciant sessió amb el teu compte de Google.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


En el següents enllaços ([enllaç 1](https://drive.google.com/drive/folders/1IlUM-jFv26QOl7ZjsoB-ShZEu7a576H8?usp=sharing), [enllaç 2](https://drive.google.com/drive/folders/19OQ_CvwHfhBYbkxCCwhW6VuM2SMabQPL?usp=sharing)) hi han les carpetes que contenen els fitxers d'entrenament. Addicionalment, també hi ha la [carpeta 3](https://drive.google.com/drive/folders/1BADFQoKME8XMoBseY1Zt6i4D32pCmsb4?usp=sharing) que conté MIDI de grups i compositors famosos. Per poder executar el codi correctament, s'ha de buscar el seu directori dins del drive (a la icona de la carpeta de la esquerra es veuen tots els fitxers i si fas click dret sobre un dells ja t'apareix la opció de copiar el directori) i copiar-lo en la variable *cancogeneral* del codi. 

## Coses a tenir en compte a la hora d'executar el codi:
- El learning rate amb aquesta versió ha de ser molt baix, cap a 0.0000001.
- No es recomana agafar menys de 100 notes d'entrada, ja que a vegades pot donar error.
- Recordar canviar el directori si està en un altre lloc del vostre Drive
- No posar un nombre excessiu de notes, neurones i repeticions, ja que tardarà molta estona a completar-se.
- Si surt un error que diu alguna cosa similar a : Is a directory: Songs/Training/.ipynb_chekcpoints, executar el codi que posa *Elimina possible error*.
- Si el fitxer no s'ha baixat bé, executar el penúltim codi


In [ ]:
# @title Codi { form-width: "50px" }
# @markdown Aquest és el codi del treball en si. Al executar-lo et demanarà una sèrie de paràmetres, com el número de notes d'entrada o el número de neurones. A la part inferior es pot anar observant la sortida de pantalla, que ens mostra dades sobre l'entrenament de la xarxa. Per veure el codi, cal simplent clicar on posa Show Code.

from google.colab import files
from random import random
import random
from random import seed
import numpy
import mido
import math
import os
import matplotlib.pyplot as plt
from mido import Message, MidiFile, MidiTrack, MAX_PITCHWHEEL, MetaMessage, UnknownMetaMessage


# cancogeneral = (input('Directori de la cançó: '))
#!!!! el directori d'aquí baix igual s'ha de canviar en funció d'on estigui la vostra carpeta del drive :)
cancogeneral = "/content/drive/MyDrive/TDR/Songs/Songs/Training"
# inputsgeneral = int(input('Nombre d'inputs: '))
inputsgeneral = 8
notesgeneral = int(input('Nombre de notes: '))
neuronesgeneral = int(input('Nombre de neurones: '))
learningrategeneral = float(input('Learning rate: ')) #recomano que sigui bastant baix, 0.00001
repsgeneral = int(input('Nombre de repeticions: '))
# sortidageneral = int(input('Nombre de notes de sortida: '))
sortidageneral = 180


class capa: #classe encarregada de crear les capes de la xarxa
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.1 * numpy.random.randn(n_inputs, n_neurons) #inicialitza els pesos i els biases aleatòriament
        self.biases = 0.005 * numpy.random.randn(1, n_neurons)

    def forward(self, inputs):
        self.output = numpy.dot(inputs, self.weights) + self.biases #passa els valors per la xarxa (multiplica els inputs pels pesos i hi suma els biases)

class ReLu: 
    def forward(self, inputs): #funció d'activació ReLu
        self.output = numpy.maximum(0, inputs)
    def backward(self, x): #derivada de la ReLu
        x[x <= 0] = 0
        x[x > 0] = 1
        self.output = x

#les classes LReLu i ELU no acaben de funcionar sempre
class LReLu:
    def forward(self, x):
        x[x<=0] = 0.1*x
        self.output = x
    def backward(self, x):
        x[x <= 0] = 0.1
        x[x > 0] = 1
        self.output = x

class ELU:
    def forward(self, x):
        for l in x:
            l[l <=0 ] = 1 * (numpy.exp(l) - 1)
        self.output = x
    def backward(self, x):
        x[x > 0] = 1
        x[x <= 0] = 1 * numpy.exp(x)
        self.output = x

class Sig: #funció d'activació sig, menys eficient que ReLu
    def forward(self, inputs):
        self.output = 1.0 / (1.0 + numpy.exp(-inputs))
    def backward(self, x):
        self.output = sigmoid(x) * (1 - sigmoid(x))


def sigmoid(x):
    return 1 / (1 + numpy.exp(-x))


class Softmax: #funció de classificació multiclasse, retorna les probabilitats per a cada classe
    def forward(self, inputs):
        self.inputs = inputs
        exp_values = numpy.exp(inputs - numpy.max(inputs, axis=1,keepdims=True))
        probabilities = exp_values / numpy.sum(exp_values, axis=1,keepdims=True)

        self.output = probabilities


# -------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------------------

def arraytomidi(fitxer): #funció que converteix un array de nombres en un fitxer MIDI
    notes = fitxer
    outfile = MidiFile()
    track = MidiTrack()
    lll = 0
    outfile.tracks.append(track)
    wa = 0
    # print(len(notes))
    ar = notes[0]
    outfile.type = int(ar[0])
    # print (int(ar[0]))
    ar = notes[1]
    outfile.ticks_per_beat = int(ar[0])
    # print(int(ar[0]))
    ar = notes[2]
    outfile.charset = (ar[0])
    # print(outfile.charset)
    ar = notes[3]
    notes = notes[3]
    # print(notes)
    ritmes = [240, 240, 240, 480, 480, 960, 1920, 240, 480, 480, 480, 960]
    for i in range(len(notes) - 1):
        i = i + 1
        ar = []
        ar.append(notes[i])
        track.append(Message('program_change', channel=0, program=1, time=0)) #cada track.append afegeix un missatge al fitxer MIDI
        track.append(MetaMessage(type="set_tempo", tempo=90000, time=0))
        if (ar != []):
            if (isinstance(ar[0], int) == True):
                if (ar[0] == 1 and len(ar) == 4):
                    track.append(Message('note_on', note=ar[2], velocity=ar[3], time=ar[4]))
                elif (ar[0] == 0 and len(ar) == 4):
                    track.append(Message('note_off', note=ar[2], velocity=ar[3], time=ar[4]))
                elif (ar[0] == 1 and len(ar) == 5):
                    track.append(Message('note_on', channel=ar[1], note=ar[2], velocity=ar[3], time=ar[4]))
                    lll = lll + 1
                elif (ar[0] == 0 and len(ar) == 5):
                    track.append(Message('note_off', channel=ar[1], note=ar[2], velocity=ar[3], time=ar[4]))
                    lll = lll + 1
                elif (len(ar) == 3 and ar[0] == 0):
                    track.append(Message('note_off', channel=0, note=ar[1], velocity=64, time=ar[2]))
                elif (len(ar) == 3 and ar[0] == 1):
                    track.append(Message('note_on', channel=0, note=ar[1], velocity=64, time=ar[2]))
                elif (len(ar) == 1):
                    track.append(Message('note_on', channel=0, note=int(ar[0]), velocity=(random.randint(20, 50)),
                                         time=random.randint(1, 20)))
                    track.append(Message('note_off', channel=0, note=int(ar[0]), velocity=(random.randint(20, 40)),
                                         time=ritmes[random.randint(0, 1)])) #si no té temps, li afegeix un temps aleatòriament
            elif (len(ar) != 1):
                e = ar[0]
                if (ar[2] == 'end_of_track'):
                    l2 = ar[3].split("=")
                    e2 = l2[1].replace('>', '')
                    track.append(MetaMessage(type='end_of_track', time=int(e2)))
                    if (i + 1 < len(notes)):
                        ar2 = notes[i + 1]
                        ee = ar2[0]
                        if (ee[2] != 'track_name'):
                            if (outfile.type != 0):
                                track = MidiTrack()
                                track.append(MetaMessage(type='track_name', name=("track" + str(wa)), time=int(e2)))
                                outfile.tracks.append(track)
                            else:
                                track.append(MetaMessage(type='track_name', name=("track" + str(wa)), time=int(e2)))
                            wa = wa + 1
                #comprova tots els possibles tipus de missatge

                elif (ar[2] == 'track_name'):
                    l2 = ar[3].split("=")
                    l3 = ar[(len(ar) - 1)].split("=")
                    e2 = l3[1].replace('>', '')
                    if (outfile.type != 0):
                        track = MidiTrack()
                        a = str(l2[1])
                        track.append(MetaMessage(type='track_name', name=a, time=int(e2)))
                        outfile.tracks.append(track)
                    else:
                        a = str(l2[1])
                        track.append(MetaMessage(type='track_name', name=a, time=int(e2)))
                elif (ar[2] == 'time_signature'):
                    l2 = ar[3].split("=")
                    l3 = ar[4].split("=")
                    l4 = ar[5].split("=")
                    l5 = ar[6].split("=")
                    l6 = ar[7].split("=")
                    e2 = l6[1].replace('>', '')
                    track.append(MetaMessage(type="time_signature", numerator=int(l2[1]), denominator=int(l3[1]),
                                             clocks_per_click=int(l4[1]), notated_32nd_notes_per_beat=int(l5[1]),
                                             time=int(e2)))
                elif (ar[2] == 'key_signature'):
                    l2 = ar[3].split("=")
                    e1 = l2[1]
                    l3 = ar[4].split("=")
                    e2 = l3[1].replace('>', '')
                    track.append(MetaMessage(type="key_signature", key=e1[1], time=int(e2)))
                elif (ar[2] == 'set_tempo'):
                    l2 = ar[3].split("=")
                    l3 = ar[4].split("=")
                    e2 = l3[1].replace('>', '')
                    track.append(MetaMessage(type="set_tempo", tempo=int(l2[1]), time=int(e2)))
                elif (ar[2] == 'instrument_name'):
                    l2 = ar[3].split("=")
                    l3 = ar[len(e) - 1].split("=")
                    e2 = l3[1].replace('>', '')
                    track.append(MetaMessage(type="instrument_name", name=l2[1], time=int(e2)))
                elif (e == 'program_change'):
                    l2 = ar[1].split("=")
                    l3 = ar[2].split("=")
                    l4 = ar[3].split("=")
                    e2 = l4[1].replace('>', '')
                    track.append(Message('program_change', channel=int(l2[1]), program=int(l3[1]), time=int(l4[1])))
                elif (e == 'control_change'):
                    l2 = ar[1].split("=")
                    l3 = ar[2].split("=")
                    l4 = ar[3].split("=")
                    l5 = ar[4].split("=")
                    track.append(Message('control_change', channel=int(l2[1]), control=int(l3[1]), value=int(l4[1]),
                                         time=int(l5[1])))
                elif (e == 'sysex'):
                    l2 = ar[1].split("=")
                    l3 = ar[2].split("=")
                    e2 = l3[1].replace('>', '')
                    e3 = l2[1].split(',')
                    e4 = []
                    for w in e3:
                        o = w.replace('(', '')
                        o = o.replace(')', '')
                        e4.append(int(o))
                    track.append(Message(type='sysex', data=e4, time=int(e2)))

                elif (ar[2] == 'midi_port'):
                    l2 = ar[3].split("=")
                    l3 = ar[4].split("=")
                    e2 = l3[1].replace('>', '')
                    track.append(MetaMessage(type='midi_port', port=int(l2[1]), time=int(e2)))
                elif (ar[2] == 'lyrics'):
                    l2 = ar[3].split("=")
                    l3 = ar[len(ar) - 1].split("=")
                    e2 = l3[1].replace('>', '')
                    track.append(MetaMessage(type="lyrics", text=l2[1], time=int(e2)))
                elif (ar[0] == 'control_change'):
                    l2 = ar[1].split("=")
                    l3 = ar[2].split("=")
                    l4 = ar[3].split("=")
                    l5 = ar[4].split("=")
                    track.append(Message(type='sysex', channel=int(l2), control=int(l3), value=int(l4), time=int(l5)))
                elif (ar[2] == 'smpte_offset'):
                    l2 = ar[3].split("=")
                    l3 = ar[4].split("=")
                    l4 = ar[5].split("=")
                    l5 = ar[6].split("=")
                    l6 = ar[7].split("=")
                    l7 = ar[8].split("=")
                    l8 = ar[9].split("=")
                    e2 = l8[1].replace('>', '')
                    track.append((MetaMessage(type='smpte_offset', frame_rate=int(l2[1]), hours=int(l3[1]),
                                              minutes=int(l4[1]), seconds=int(l5[1]), frames=int(l6[1]),
                                              sub_frames=int(l7[1]), time=int(e2))))
                else: 
                    res = "res"
                    # print("CIGUEÑA")
                    # print(ar)

    outfile.save('/content/drive/MyDrive/TDR/Songs/Songs/prova.mid') #guarda el fitxer


def miditoarray(midi): #funció encarregada de transformar els MIDI en un array
    if __name__ == '__main__':
        w = 0
        mat = []
        arr = []
        midi_file = MidiFile(midi)
        arr.append(midi_file.type)
        mat.append(arr)
        arr = []
        arr.append(midi_file.ticks_per_beat)
        mat.append(arr)
        arr = []

        arr.append(midi_file.charset)
        mat.append(arr)
        arr = []
        # print(midi_file.ticks_per_beat)
        # print(midi_file.type)
        i = 0
        for i, track in enumerate(midi_file.tracks):
            if (i == 2):
                break
            i = i + 1
            # print("tratratra")

            for message in track: #itera per tots els missatges

                e = (format(message).split())
                if (e[0] == 'program_change'):
                    arr.append(e)
                elif (e[0] == '<meta'):
                    arr.append(e)

                elif (e[0] == 'pitchwheel'):
                    l1 = (e[1].split('=')) #separa el missatge pel signe =
                    l2 = (e[2].split('='))
                    l3 = (e[3].split('='))
                    arr.append(int(l1[1])) #afegeix només el nombre que hi ha després del =
                    arr.append(int(l2[1]))
                    arr.append(int(l3[1]))
                elif (e[0] == 'note_on' or e[0] == 'note_off'):

                    if (e[0] == "note_on"):
                        arr.append(1)
                    else:
                        arr.append(0)
                    l1 = (e[1].split('='))
                    l2 = (e[2].split('='))
                    l3 = (e[3].split('='))
                    l4 = (e[4].split('='))
                    arr.append(int(l1[1]))
                    arr.append(int(l2[1]))
                    arr.append(int(l3[1]))
                    arr.append(int(l4[1]))
                elif (e[0] == 'control_change'):
                    arr.append(e)
                elif (e[0] == 'sysex'):
                    arr.append(e)
                if (arr != []):
                  mat.append(arr)

                arr = []

                w = w + 1
    else:
        res = "res"
        # print("aveurequepasa")
    
    return mat #retorna una matriu amb totes les notes i els metamissatges


def graficador(s): #crea una gràfica a partir d'un array
    nota = [] #crea una llista de notes
    temps = [] #crea una llista de temps 
    melo = []
    oo = []
    onota = []
    otemps = []
    matriu = []
    meta = []
    i = 0
    e = 0
    oe = 0
    for c in s:
        if (len(c) != 0 and len(c) != 1 and len(c) == 5):
            oo.append(c[0])
            if (c[0] == 1):
                nota.append(c[2]) #afegeix la nota a la llista de notes
                if (temps != []):
                    temps.append(c[4] + temps[e]) #afegeix el temps a la llista de temps
                    e = e + 1
                else:
                    temps.append(c[4] + 0)

            elif (c[0] == 1 and c[4] != 0):
                w = 0
                while (w < c[4]):
                    melo.append(c[2])
                    w = w + 1
            elif (c[0] == 0):
                onota.append(c[2])
                if (otemps != []):
                    otemps.append(c[4] + otemps[oe])
                    oe = oe + 1
                else:
                    otemps.append(c[4] + 0)

        elif (len(c) != 0 and isinstance(c[0], int) == False and c[0] != "latin1"):
            oo.append(2)
            a = c[0]
            meta.append(a)
    
    plt.plot(temps, nota) #crea el gràfic utilitzant les llistes temps i nota
    plt.title('Test 1')
    plt.xlabel('Temps')
    plt.ylabel('Nota')
    plt.show() 
    plt.savefig('V2-generacio.png')
    matriu.append(s[0])
    matriu.append(s[1])
    matriu.append(s[2])
    i = 0
    l = 0
    p = 0
    #part del codi que realment no té cap funcionalitat en aquesta versió, utilitzada en les primeres versions
    for y in oo:
        if (y == 0):
            if (l != 0):
                matriu.append([y, onota[l], otemps[l] - otemps[l - 1]])
            elif (onota != [] and otemps != []):
                matriu.append([y, onota[l], otemps[l]])
            l = l + 1
        elif (y == 1):
            if (i != 0):
                matriu.append([y, nota[i], temps[i] - temps[i - 1]])
            elif (nota != [] and temps != []):
                matriu.append([y, nota[i], temps[i]])
            i = i + 1
        elif (y == 2):
            matriu.append(meta[p])
            p = p + 1
    return (nota) #retorna només les notes
    


def grafcomparacio(s1, s2): #gràfic que compara dos arrays
    nota1 = []
    temps1 = []
    melo1 = []
    nota2 = []
    temps2 = []
    melo2 = []
    oo = []
    onota = []
    otemps = []
    matriu = []
    meta = []
    i = 0
    e = 0
    oe = 0
    for c in s1: #fa el mateix que la funció graficador
        if (len(c) != 0 and len(c) != 1 and len(c) == 5):
            oo.append(c[0])
            if (c[0] == 1):
                nota1.append(c[2])
                if (temps1 != []):
                    temps1.append(c[4] + temps1[e])
                    e = e + 1
                else:
                    temps1.append(c[4] + 0)

            elif (c[0] == 1 and c[4] != 0):
                w = 0
                while (w < c[4]):
                    melo1.append(c[2])
                    w = w + 1
            elif (c[0] == 0):
                onota.append(c[2])
                if (otemps != []):
                    otemps.append(c[4] + otemps[oe])
                    oe = oe + 1
                else:
                    otemps.append(c[4] + 0)

        elif (len(c) != 0 and isinstance(c[0], int) == False and c[0] != "latin1"):
            oo.append(2)
            a = c[0]
            meta.append(a)
    i = 0
    e = 0
    oe = 0
    for c in s2: #repeteix el mateix per l'altre array
        if (len(c) != 0 and len(c) != 1 and len(c) == 5):
            oo.append(c[0])
            if (c[0] == 1):
                nota2.append(c[2])
                if (temps2 != []):
                    temps2.append(c[4] + temps2[e])
                    e = e + 1
                else:
                    temps2.append(c[4] + 0)

            elif (c[0] == 1 and c[4] != 0):
                w = 0
                while (w < c[4]):
                    melo2.append(c[2])
                    w = w + 1
            elif (c[0] == 0):
                onota.append(c[2])
                if (otemps != []):
                    otemps.append(c[4] + otemps[oe])
                    oe = oe + 1
                else:
                    otemps.append(c[4] + 0)

        elif (len(c) != 0 and isinstance(c[0], int) == False and c[0] != "latin1"):
            oo.append(2)
            a = c[0]
            meta.append(a)

    plt.plot(temps1, nota1, 'b.-', label="Original")
    plt.plot(temps1[:len(nota2)], nota2, 'm.-', label="Predicció") #crea les dues gràfiques una sobre l'altra
    plt.title('Test 1')
    plt.xlabel('Temps')
    plt.ylabel('Nota')
    plt.show()
    plt.savefig('V2-entrenament.png')

def dades(array): #estructura les dades
    i = 0
    matriu = []
    pred = []
    while (i < (len(array) - inputsgeneral)):
        nou = []
        l = 0
        while (l < inputsgeneral): #agrupa les dades en grups de llargada inputsgeneral
            nou.append(array[l + i]) 
            l = l + 1
        matriu.append(nou)
        pred.append(array[i + inputsgeneral]) #afegeix la següent nota
        i = i + 1
    print(matriu)
    print("prediccions")
    print(pred)
    return matriu, pred #retorna la matriu de notes i la llista amb les notes següents (X i Y)


# -------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------------------
ar = []
for l in os.listdir(cancogeneral): #itera per tots els elements de la carpeta
    l = cancogeneral + "/" + l
    arr = miditoarray(l) #transforma els midis en un array que conté totes les cançons
    ar = ar + arr

X, y = dades(graficador(ar[:notesgeneral])) #crida les funcions graficador i dades
X = numpy.array(X[:notesgeneral]) #agafa només les primeres 'notesgeneral' notes
y = numpy.array(y[:notesgeneral]) 
# print(y)
# Create model
dense1 = capa(inputsgeneral, neuronesgeneral)  # crea la primera capa (la capa d'inputs no conta)
activation1 = ReLu() #funció d'activació de la primera capa
dense2 = capa(neuronesgeneral, 127)  # crea la segona capa
activation2 = Softmax() #funció d'activació de la segona capa

lowest_loss = 9999999  # valor inicial del valor més petit, que posteriorment s'anirà rebaixant
best_dense1_weights = dense1.weights.copy()
best_dense1_biases = dense1.biases.copy()
best_dense2_weights = dense2.weights.copy()
best_dense2_biases = dense2.biases.copy()

one_hot_y = numpy.zeros((len(y), 127)) #variable que conté els valors de y one-hot encoded
#print(one_hot_y)
u = 0
for e in y:
    one_hot_y[u, e - 1] = 1
    u = u + 1
for iteration in range(repsgeneral): #repeteix tantes vegades com repsgeneral
    #print(X)
    #print(repsgeneral)
    dense1.forward(X) #realitza un forward pass per totes les capes
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)

    y_pred_clipped = numpy.clip(activation2.output, 1e-7, 1 - 1e-7) #evita el logaritme de 0
    loss = numpy.sum(-one_hot_y * numpy.log(y_pred_clipped)) #calcula el loss
    loss = numpy.mean(loss) 
    predictions = numpy.argmax(activation2.output, axis=1) 
    accuracy = numpy.mean(predictions == y) #calcula l'encert
    if loss < lowest_loss:
        best_dense1_weights = dense1.weights.copy() #si el loss és més petit que el loss més petit, guarda els pesos com a els millors pesos
        best_dense1_biases = dense1.biases.copy()
        best_dense2_weights = dense2.weights.copy()
        best_dense2_biases = dense2.biases.copy()
        lowest_loss = loss
    if iteration%100 == 0:
        print('Current set of weights, iteration:', iteration, 'loss:', loss, 'acc:', accuracy) #mostra per pantalla els valors cada 100 iteracions

    #retropropagació (càlcul del gradient)
    dloss_dsum = activation2.output - one_hot_y
    dsum_dpes = activation1.output

    dloss_pes = numpy.dot(dsum_dpes.T, dloss_dsum)
    dloss_bias = dloss_dsum

    dsum_dah = dense2.weights
    dloss_dah = numpy.dot(dloss_dsum, dsum_dah.T)

    activation1.backward(dense1.output)

    dah_dsumh = activation1.output
    dsumh_dpesh = X
    dloss_pesh = numpy.dot(dsumh_dpesh.T, dah_dsumh * dloss_dah)
    dloss_biash = dloss_dah * dah_dsumh

    #actualitza els pesos

    dense1.weights -= learningrategeneral * dloss_pesh
    dense1.biases -= learningrategeneral * dloss_biash.sum(axis=0)

    dense2.weights -= learningrategeneral * dloss_pes
    dense2.biases -= learningrategeneral * dloss_bias.sum(axis=0)
print('Lowest loss:', lowest_loss)
dense1.weights = best_dense1_weights.copy()
dense1.biases = best_dense1_biases.copy()
dense2.weights = best_dense2_weights.copy()
dense2.biases = best_dense2_biases.copy()
    

dense1.forward(X) #realitza un forward pass amb els millors pesos
activation1.forward(dense1.output)
dense2.forward(activation1.output)
activation2.forward(dense2.output)

array = [[0], [96], ['latin1']]
eee = numpy.argmax(activation2.output, axis=1).tolist() #converteix la sortida en una llista de notes
array.append(eee)
arraytomidi(array) #converteix la llista en un fitxer
e = miditoarray("/content/drive/MyDrive/TDR/Songs/Songs/prova.mid")
print(e)
grafcomparacio(ar[:notesgeneral], e) #compara les dades inicials amb les prediccions

ar = miditoarray('/content/drive/MyDrive/TDR/Songs/Songs/inici.mid') 
X, y = dades(graficador(ar[:notesgeneral+1])) #X passa a ser les notes d'inici
X = numpy.array(X[:notesgeneral])
y = numpy.array(y[:notesgeneral])

k = 0
kl = []
array2 = [[0], [96], ['latin1']]
X = X[0]
for c in X:
    kl.append(int(c))
while (k < sortidageneral):
    dense1.forward(X) #realitza un forward pass
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)
    k = k + 1
    eee2 = numpy.argmax(activation2.output, axis=1).tolist()
    eee2 = eee2[0] #agafa la predicció
    kl.append(eee2)
    X = X.tolist()
    X.append(eee2) #afegeix la predicció a X
    X.pop(0) #elimina el primer element de X
    X = numpy.array(X)
for i in kl:
    i = int(i)
array2.append(kl) 

# print(array2)
arraytomidi(array2[:sortidageneral]) #converteix a MIDI la cançó generada
graficador(miditoarray("/content/drive/MyDrive/TDR/Songs/Songs/prova.mid")) #mostra la gràfica de la cançó
files.download("/content/drive/MyDrive/TDR/Songs/Songs/prova.mid") #descarrega la cançó

In [ ]:
#@title Exporta la cançó generada
from google.colab import files &> /dev/null
files.download( "/content/drive/MyDrive/TDR/Songs/Songs/prova.mid" ) &> /dev/null
print("Fet! 🥳")

In [ ]:
#@title Elimina possible error
rmdir /content/drive/MyDrive/TDR/Songs/Songs/Training/.ipynb_checkpoints
#Si el vostre directori d'entrenment es diferent, canvieu-lo aquí també

## **Part No-TDR**
Aquesta part del codi no és ni tan sols mencionada en el tdr. L'he creat bàsicament per facilitar escoltar els fitxers MIDI. Bàsicament, és un conversor MIDI - .wav.

In [ ]:
#@title Conversor MIDI - wav
#@markdown Instal·la les llibreries necessàries i converteix el fitxer MIDI en un fitxer wav.
!apt install fluidsynth &> /dev/null
!cp /content/drive/MyDrive/TDR/Songs/Songs/Kawai Grand Piano.sf2 ./font.sf2 &> /dev/null
!fluidsynth -ni /usr/share/sounds/sf2/FluidR3_GM.sf2 /content/drive/MyDrive/TDR/Songs/Songs/prova.mid -F /content/drive/MyDrive/TDR/Songs/Songs/wavprova.wav -r 44100 &> /dev/null
print("Fet! 🥳")



Fet! 🥳


In [ ]:
#@title Exporta la cançó generada en format àudio
from google.colab import files
files.download( "/content/drive/MyDrive/TDR/Songs/Songs/wavprova.wav" ) 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Escolta la melodia
#@markdown Permet reproduïr el fitxer wav directament des d'aquí.
#@markdown  
#@markdown <font color="red">⚠ No sempre funciona (sovint es desconecta el Colab)
from IPython.display import Audio
Audio('/content/drive/MyDrive/TDR/Songs/Songs/wavprova.wav') 

In [ ]:
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc
!cp drive/My Drive/Colab Notebooks/Untitled.ipynb ./
!jupyter nbconvert --to PDF "Untitled.ipynb"
jupyter nbconvert --execute --to pdf notebook.ipynb